In [ ]:
from moonbox import *

In [ ]:
ods = [parse_oneday(get_oneday(date=f"2024-03-{x}")) for x in range(1, 5)]
ods

In [ ]:
# total area: pi*R^2
# foreground: 0.5*pi*R^2 + 0.5*pi*R*r = 0.5*pi*R^2*(1+r/R)
# f = 0.5*(1+r/R)
# R*(2*f-1)=r

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches

def draw_moon(axes, f, direction):
    center = (0, 0)
    radius = 0.9

    axes.set(aspect=1, xlim=(-1.0, 1.0), ylim=(-1.0, 1.0))
    axes.set_axis_off()

    if f == 0.0:
        back_color = "black"
        half_side = None
        half_color = None
        ellipse_color = None
    elif direction == "waxing" and 0.0 < f < 0.5:
        back_color = "white"
        half_side = "left"
        half_color = "black"
        ellipse_color = "black"
    elif direction in ["waxing", "first"] and f == 0.5:
        back_color = "white"
        half_side = "left"
        half_color = "black"
        ellipse_color = None
    elif direction == "waxing" and 0.5 < f < 1.0:
        back_color = "black"
        half_side = "right"
        half_color = "white"
        ellipse_color = "white"
    elif f == 1.0:
        back_color = "white"
        half_side = None
        half_color = None
        ellipse_color = None
    elif direction == "waning" and 0.5 < f < 1.0:
        back_color = "black"
        half_side = "left"
        half_color = "white"
        ellipse_color = "white"
    elif direction in ["waning", "third"] and f == 0.5:
        back_color = "black"
        half_side = "left"
        half_color = "white"
        ellipse_color = None
    elif direction == "waning" and 0.0 < f < 0.5:
        back_color = "white"
        half_side = "right"
        half_color = "black"
        ellipse_color = "black"
    else:
        raise RuntimeError(f"bad values: f={f} direction={direction}")

    back = mpatches.Circle(center, radius, ec="none")
    back.set(color=back_color)
    axes.add_artist(back)

    if half_side == "left":
        half = mpatches.Wedge(center, radius, 90, 270, ec="none")
        half.set(color=half_color)
        axes.add_artist(half)
    elif half_side == "right":
        half = mpatches.Wedge(center, radius, 270, 90, ec="none")
        half.set(color=half_color)
        axes.add_artist(half)

    if ellipse_color is not None:
        artist = mpatches.Ellipse(center, 2*radius * (2*f-1), 2*radius)
        artist.set(color=ellipse_color)
        axes.add_artist(artist)

    return None

n_plots = 16
fig, axs = plt.subplots(1, n_plots)
for i, ax in enumerate(axs):
    phase = i / n_plots

    if 0.0 <= phase <= 0.5:
        f = 2 * phase
        direction = 'waxing'
    elif 0.5 < phase <= 1.0:
        f = 1.0 - 2.0 * (phase - 0.5)
        direction = 'waning'

    draw_moon(axs[i], f, direction)

fig.set_facecolor("0.5")
plt.show()

In [ ]:
days = [{"f": x['illumination'] / 100, "direction": x['phase'].split()[0].lower()} for x in ods]

n_plots = len(days)
fig, axs = plt.subplots(1, n_plots, figsize=(12, 1))
for i, ax in enumerate(axs):
    draw_moon(axs[i], **days[i])

fig.set_facecolor("0.5")
plt.show()

## Plot all the dates in a year

In [ ]:
data = get_oneday_year()
data[0]

In [ ]:

n_months = 12
fig, axs = plt.subplots(n_months, 31, figsize=(31, n_months))
for i in range(n_months):
    month = i + 1
    days = [{"f": x['illumination'] / 100, "direction": x['phase'].split()[0].lower()} for x in data if x["date"].month == month]
    for j in range(len(days)):
        draw_moon(axs[i, j], **days[j])

    if len(days) < 31:
        for j in range(len(days), 31):
            axs[i, j].set_visible(False)

fig.set_facecolor("0.5")
plt.show()